#Ajuste fino do conjunto de dados CSTNews usando BERT Transformers by HuggingFace e Lotes Inteligentes e Validação Cruzada para todos os Folds

Com base no modelo BERT pré-treinado realiza o ajuste fino para os dados CSTNews para classificar documentos coerentes e incoerentes.

- Realiza o ajuste fino nos dados de CSTNEWS de Márcio Dias.
- Realiza a avaliação do conjunto de teste para um determinado fold(10%).
- A seção 2 - parametrização define os argumentos da execução.

Notebook base: https://colab.research.google.com/drive/1KDeFAHvRFq3bY5onzj8Wgul3rRcrQNaC?usp=sharing

----------------------------

**Link biblioteca Transformers:**
https://github.com/huggingface/transformers


**Artigo original BERT:**
https://arxiv.org/pdf/1506.06724.pdf

**Artigo padding dinâmico:**
https://towardsdatascience.com/divide-hugging-face-transformers-training-time-by-2-or-more-21bf7129db9q-21bf7129db9e

# 1 Preparação do ambiente
Preparação do ambiente para execução do notebook.

## 1.1 Tempo inicial de processamento

In [1]:
# Import das bibliotecas
import time
import datetime

# Marca o tempo de início do processamento
inicioProcessamento = time.time()

In [2]:
print("  Tempo de início de processamento:  {:} (h:mm:ss)".format(inicioProcessamento))

  Tempo de início de processamento:  1634027794.5626886 (h:mm:ss)


## 1.2 Tratamento de logs

In [3]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## 1.3 Identificando o ambiente Colab

In [4]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = 'google.colab' in sys.modules

## 1.4 Biblioteca de limpeza de tela

In [5]:
from IPython.display import clear_output

## 1.5 Conecta ao Google Drive

É necessário existir a pasta '/content/drive/MyDrive/Colab Notebooks/Data/CSTNEWS/Resultados/' para receber os resutlados do notebook.

In [6]:
# Monta o Google Drive para esta instância de notebook.
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## 1.6 Instalação do wandb

Instalação

In [7]:
!pip install --upgrade wandb

     |████████████████████████████████| 1.7 MB 12.8 MB/s 
     |████████████████████████████████| 97 kB 6.6 MB/s 
     |████████████████████████████████| 139 kB 49.2 MB/s 
     |████████████████████████████████| 180 kB 49.0 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=e6bb879ec3f1eab9433063e378fcf2761a3de0c1714ac223cd412fdaa060ced2
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=d93d866c628a9268d38eb0ffc67fc8c7e4bc21b7cd14a0b4da5cd9f5d6831f30
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


## 1.7 Instalação BERT da Hugging Face

Instala a interface pytorch para o BERT by Hugging Face. 

In [8]:
!pip install -U transformers==4.5.1

     |████████████████████████████████| 2.1 MB 12.8 MB/s 
     |████████████████████████████████| 895 kB 40.6 MB/s 
     |████████████████████████████████| 3.3 MB 49.9 MB/s 


## 1.8 Recupera o cohebert do Github

In [9]:
!git clone https://github.com/osmarbraz/cohebert_v1.git

Cloning into 'cohebert_v1'...
remote: Enumerating objects: 4786, done.
remote: Counting objects: 100% (418/418), done.
remote: Compressing objects: 100% (270/270), done.
remote: Total 4786 (delta 283), reused 260 (delta 143), pack-reused 4368
Receiving objects: 100% (4786/4786), 28.78 MiB | 10.51 MiB/s, done.
Resolving deltas: 100% (3037/3037), done.


In [10]:
#Muda o diretório corrente para a pasta clonada
import sys
sys.path.append('./cohebert_v1/cohebert')

In [11]:
# Import de bibliotecas
from util.utilmodulo import *
from util.utiltempo import *
from util.utilarquivo import *

## 1.9 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [12]:
# Importando a biblioteca
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == '/device:GPU:0':
    print('Encontrei GPU em: {}'.format(device_name))
else:
    print('Dispositivo GPU não encontrado')
    #raise SystemError('Dispositivo GPU não encontrado')

Encontrei GPU em: /device:GPU:0


Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos os dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla V100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla V100-SXM2-16GB(Pro)
- 2o Tesla P100-PCIE-16GB
- 3o Tesla T4
- 4o Tesla P4 (Não tem memória para execução 4 lotes de treino x 8 lotes de avaliação, somente 2 x 4)
- 5o Tesla K80 (Não tem memória para execução 4 lotes de treino x 8 lotes de avaliação, somente 2 x 4)

In [13]:
# Biblioteca cohebert
from bert.bertmodulo import *

device = getDeviceGPU()

2021-10-12 08:38:12,830 : INFO : Existem 1 GPU(s) disponíveis.
2021-10-12 08:38:27,886 : INFO : Iremos usar a GPU: Tesla K80.


# 2 Parametrização

In [14]:
# Biblioteca cohebert
from transformers import TrainingArguments

# Definição dos parâmetros de Treinamento.
training_args = TrainingArguments(
    # AjusteFinoCSTNews_v1_C_SB_KF = nome do notebook
    # E = número de épocas
    # lr = taxa de aprendizagem
    # b = lotes de treino e avaliação
    # f = número do fold
    output_dir = 'cohebert_AjusteFinoCSTNews_v1_C_SB_KF_E_4_lr_5_b_4_8_f',    
    save_steps = 0,    
    seed = 42,
    num_train_epochs = 1, # Intervalo de valores: 2, 3, 4
    learning_rate = 1e-5, # Intervalo de valores: 1e-5, 2e-5, 3e-5, 4e-5, 5e-5 
    gradient_accumulation_steps = 1,
    per_device_train_batch_size = 4, 
    per_device_eval_batch_size = 8,        
    evaluation_strategy = 'epoch'
)

# Import de bibliotecas
from bert.bertarguments import ModeloArgumentosClassificacao

# Definição dos parâmetros do Modelo
model_args = ModeloArgumentosClassificacao(     
    max_seq_len = 512,
    #pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip",
    pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip",    
    #pretrained_model_name_or_path = 'bert-base-multilingual-cased',
    do_lower_case = False,   # default True
    num_labels = 2,
    output_attentions = False,    # default False
    output_hidden_states = False, # default False 
    optimizer = 'AdamW',
    use_wandb = False,
    salvar_modelo_wandb = False,
    salvar_modelo = False,
    salvar_avaliacao = True, # Salva o resultado classificações
    salvar_classificacao = True, # Salva o resultado da avaliação das classificações
    fold = 1 # Intervalo de valores: 1 a 10, É utilizado somente para logs. Use as variáveis das últimas linhas de bloco para definir o intervalo de folds a serem gerados.
)

# Determina o intervalo de folds a ser avaliado
inicioFold = 1
fimFold = 10

# 3 BERT

## Carrega o modelo e tokenizador BERT

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **'neuralmind/bert-base-portuguese-cased'**
* **'neuralmind/bert-large-portuguese-cased'**

In [15]:
# Import de bibliotecas
from bert.bertmodulo import *

# Carrega o modelo e tokenizador do BERT
# model, tokenizer = carregaBERT(model_args)

# O BERT será carregado para cada fold

# 4.Wandb

https://wandb.ai/osmar-braz/ajustefinocstnews_v1_c_sb_kfold/table?workspace=user-osmar-braz


## Função de inicialização wandb

In [16]:
def inicializacaoWandb():

  if model_args.use_wandb:

    # Importando a biblioteca.
    import wandb

    #Login via linha de comando
    !wandb login aded3bc0ea651fff536cc08ba69caf8ac4141cfd

    # Inicializando o registro do experimento.
    # Na execução só pode existir de um init  para que não gere dois registros no wandb.
    wandb.init(project="ajustefinocstnews_v1_c_sb_kfold", name=training_args.output_dir + str(model_args.fold))

    # Atualiza os parâmetros do modelo no wandb.
    wandb.config.update(model_args)
    # Atualiza os parâmetros de treinamento no wandb.
    wandb.config.update(training_args)

    # Registra os parämetros não literais do model_args.
    wandb.log({"max_seq_len": model_args.max_seq_len})
    wandb.log({"do_lower_case": model_args.do_lower_case})
    wandb.log({"output_hidden_states": model_args.output_hidden_states})    
    wandb.log({"fold": model_args.fold})

    return wandb

# 5 Treinamento e avaliação

Treinando e avaliando o conjunto de dados.

## 5.1 Execução do treinamento e avaliação de todos os Folds

In [17]:
# Import das bibliotecas
import time
import datetime

# Biblioteca cohebert
from bert.bertmodulo import *
from conjuntodedados.dadoscstnewsclassificacaokfold import *
from experimento.classificacaobinaria  import *
from experimento.classificacaobinariakfold import *

# Percorre todos os folds do intervalo de inicioFold até fimFold
for ifold in range(inicioFold, fimFold+1):

  # Seta o parâmetro do fold
  model_args.fold = ifold
  print("\nProcessamendo do fold: {}".format(model_args.fold))
  
  # Marca o tempo de início do processamento
  inicioProcessamentof = time.time()

  print("  Tempo de início de processamento fold:  {:} (h:mm:ss)".format(formataTempo(inicioProcessamentof)))

  # Carrega o modelo e tokenizador do BERT
  model, tokenizer = carregaBERT(model_args)

  # Inicializa o wandb para registro
  wandb = inicializacaoWandb()

  # Função de carregamento dos dados de um fold
  dfdados_train, dfdados_test = getConjuntoDeDadosClassificacaoFold(model_args, tokenizer, ORIGEM=None)

  # Pega as listas de documentos de treino e seus rótulos.
  documentos_treino = dfdados_train.documento.values
  classes_treino = dfdados_train.classe.values
  documentoids_treino = dfdados_train.id.values

  # Pega as listas de documentos teste e seus rótulos.
  documentos_teste = dfdados_test.documento.values
  classes_teste = dfdados_test.classe.values
  documentoids_teste = dfdados_test.id.values

  # Mostra o resultado dos dados carregados.
  print("Total do conjunto de dados          : {}.".format(len(documentos_treino) + len(documentos_teste)))
  print("Total do conjunto de dados de treino: {}.".format(len(documentos_treino)))
  print("Total do conjunto de dados de teste : {}.".format(len(documentos_teste)))

  #################  Treinamento

  # Registra o tempo inicial.
  treinamento_t0 = time.time()

  # Realiza o treinamento.
  model = realizaTreinamento(model_args, training_args, model, tokenizer, documentos_treino, classes_treino, documentoids_treino, wandb)
  
  # Medida de quanto tempo levou a execução do treinamento.
  treinamento_total = formataTempo(time.time() - treinamento_t0)

  print("  Tempo total treinamento       : {:}".format(treinamento_total))
  
  #################  Treinamento

  ################# Avaliação

  # Registra o tempo inicial.
  avaliacao_t0 = time.time()

  # Realiza a avaliação do modelo.
  media_test_loss, acc, rec, pre, f1, vp_s, vn_s, fp_s, fn_s, lista_resultado_avaliacao = realizaAvaliacao(model_args, training_args, model, tokenizer, documentos_teste, classes_teste, documentoids_teste, wandb)

  print('Avaliação loss: {:.8f}; Acc: {:.8f}; Rec: {:.8f}; Pre: {:.8f}, F1:{:.8f}, vp: {:3d}; vn: {:3d}; fp: {:3d}; fn: {:3d}'.format( 
        media_test_loss, acc, rec, pre, f1, vp_s, vn_s, fp_s, fn_s))      
    
  print("Acurácia do fold {}        : {:.8f}".format(model_args.fold, acc))  

  # Medida de quanto tempo levou a execução do treinamento e avaliação
  avaliacao_total = formataTempo(time.time() - avaliacao_t0)

  print("Tempo gasto na avaliação  : {:}".format(avaliacao_total))

  ################# Avaliação

  # Salva o resultado da classificação  
  # Diretório para salvar o arquivo de classificação.
  DIRETORIO_CLASSIFICACAO = "/content/drive/MyDrive/Colab Notebooks/Data/CSTNEWS/validacao_classificacao/kfold/Classificacao/"
  salvaResultadoClassificacao(model_args, training_args, DIRETORIO_CLASSIFICACAO, lista_resultado_avaliacao)
  
  # Salva o resultado da avaliação
  # Diretório para salvar o arquivo.
  DIRETORIO_AVALIACAO = "/content/drive/MyDrive/Colab Notebooks/Data/CSTNEWS/validacao_classificacao/kfold/Avaliacao/"  
  salvaResultadoAvaliacao(model_args, training_args, DIRETORIO_AVALIACAO, acc, rec, pre, f1, vp_s, vn_s, fp_s, fn_s, treinamento_total)
  
  # Apaga os dados
  del dfdados_train
  del dfdados_test
  del documentos_treino
  del classes_treino
  del documentoids_treino
  del documentos_teste
  del classes_teste
  del documentoids_teste
  del lista_resultado_avaliacao  
  del model

  # Pega o tempo atual menos o tempo do início do processamento.
  finalProcessamentof = time.time()
  tempoTotalProcessamentof = formataTempo(finalProcessamentof - inicioProcessamentof)

  print("")
  print("Tempo processamento fold: {:} (h:mm:ss)".format(tempoTotalProcessamentof))


Processamendo do fold: 1
  Tempo de início de processamento fold:  18912 days, 8:38:29 (h:mm:ss)


2021-10-12 08:38:29,290 : INFO : Download do arquivo: bert-base-portuguese-cased_pytorch_checkpoint.zip.


  0%|          | 0.00/406M [00:00<?, ?B/s]

2021-10-12 08:38:48,756 : INFO : Download do arquivo: cohebert_v1/modeloBERT/vocab.txt.


  0%|          | 0.00/210k [00:00<?, ?B/s]

...

  Tempo total treinamento       : 0:15:19


2021-10-12 11:27:16,922 : INFO : Tokenização concluída.
2021-10-12 11:27:16,926 : INFO :        996 amostras.
2021-10-12 11:27:16,930 : INFO :        996 amostras após classificação.
2021-10-12 11:27:16,932 : INFO : Criando lotes de tamanho 8.
2021-10-12 11:27:16,936 : INFO : Lote criado - Selecionado 125 lotes.


Lotes :   0%|          | 0/125 [00:00<?, ?lotes/s]

2021-10-12 11:27:46,840 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/CSTNEWS/validacao_classificacao/kfold/Classificacao/.
2021-10-12 11:27:46,854 : INFO : Criando arquivo classificação: /content/drive/MyDrive/Colab Notebooks/Data/CSTNEWS/validacao_classificacao/kfold/Classificacao/cohebert_AjusteFinoCSTNews_v1_C_SB_KF_E_4_lr_5_b_4_8_f10_BERTimbau_base.csv.
2021-10-12 11:27:46,882 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/CSTNEWS/validacao_classificacao/kfold/Avaliacao/.
2021-10-12 11:27:46,890 : INFO : Criando arquivo resultado: /content/drive/MyDrive/Colab Notebooks/Data/CSTNEWS/validacao_classificacao/kfold/Avaliacao/cohebert_AjusteFinoCSTNews_v1_C_SB_KF_E_4_lr_5_b_4_8_f10_BERTimbau_base.csv.


Avaliação loss: 0.16585237; Acc: 0.95983936; Rec: 0.99196787; Pre: 0.93207547, F1:0.96108949, vp: 494; vn: 462; fp:  36; fn:   4
Acurácia do fold 10        : 0.95983936
Tempo gasto na avaliação  : 0:00:33

Tempo processamento fold: 0:16:53 (h:mm:ss)


### Carrega e calcula a acurácia médias das execuções


In [18]:
# Biblioteca cohebert
from experimento.classificacaobinariakfold import *

# Diretório para salvar o arquivo.
DIRETORIO_AVALIACAO = "/content/drive/MyDrive/Colab Notebooks/Data/CSTNEWS/validacao_classificacao/kfold/Avaliacao/"

carregaResultadoAvaliacao(model_args, training_args, DIRETORIO_AVALIACAO)

2021-10-12 11:27:46,923 : INFO : Média dos arquivos: cohebert_AjusteFinoCSTNews_v1_C_SB_KF_E_4_lr_5_b_4_8_f X _base.
2021-10-12 11:27:46,941 : INFO : Arquivo: cohebert_AjusteFinoCSTNews_v1_C_SB_KF_E_4_lr_5_b_4_8_f1_BERTimbau_base, Data: 18/09/2021 00:21, Tempo: 0:14:20, QtdeTeste: 996, Acc: 0.95080321, Rec: 0.97389558, Pre: 0.93090211, F1:0.95191364, vp:  485; vn:  462; fp:   36; fn:   13
2021-10-12 11:27:46,944 : INFO : Arquivo: cohebert_AjusteFinoCSTNews_v1_C_SB_KF_E_4_lr_5_b_4_8_f1_BERTimbau_base, Data: 21/09/2021 02:16, Tempo: 0:15:04, QtdeTeste: 996, Acc: 0.92771084, Rec: 0.98192771, Pre: 0.88586957, F1:0.93142857, vp:  489; vn:  435; fp:   63; fn:    9
2021-10-12 11:27:46,946 : INFO : Arquivo: cohebert_AjusteFinoCSTNews_v1_C_SB_KF_E_4_lr_5_b_4_8_f1_BERTimbau_base, Data: 12/10/2021 08:55, Tempo: 0:15:18, QtdeTeste: 996, Acc: 0.96084337, Rec: 1.00000000, Pre: 0.92737430, F1:0.96231884, vp:  498; vn:  459; fp:   39; fn:    0
2021-10-12 11:27:46,957 : INFO : Arquivo: cohebert_AjusteF

# 6 Finalização

## 6.1 Salvando o Modelo para o wandb

In [19]:
if model_args.use_wandb and model_args.salvar_modelo_wandb:
  
    # Salva o modelo para o wandb    
    torch.save(model.state_dict(), os.path.join(wandb.run.dir, 'model_dict.pt'))

## 6.2 Salvando o Modelo Ajustado

Grava o modelo e o tokenizador no disco.

In [20]:
def salvaModelo(model_args):
  
    if model_args.salvar_modelo:
  
        # Import de bibliotecas.
        import os

        # Salvando as melhores práticas: se você usar nomes padrão para o modelo, você pode recarregá-lo usando from_pretrained ()

        # Diretório de salvamento do modelo.
        DIRETORIO_LOCAL_MODELO_AJUSTADO = '/content/modelo_ajustado/'

        # Cria o diretório de saída se necessário.
        if not os.path.exists(DIRETORIO_LOCAL_MODELO_AJUSTADO):
            os.makedirs(DIRETORIO_LOCAL_MODELO_AJUSTADO)

        print('Salvando o modelo para {}'.format(DIRETORIO_LOCAL_MODELO_AJUSTADO))

        # Salve um modelo treinado, configuração e tokenizer usando `save_pretrained ()`.
        # Eles podem então ser recarregados usando `from_pretrained ()`.
        model_to_save = model.module if hasattr(model, 'module') else model  # Cuide do treinamento distribuído/paralelo
        model_to_save.save_pretrained(DIRETORIO_LOCAL_MODELO_AJUSTADO)
        tokenizer.save_pretrained(DIRETORIO_LOCAL_MODELO_AJUSTADO)

        # Boa prática: salve seus argumentos de treinamento junto com o modelo treinado.
        torch.save (model_args, os.path.join (DIRETORIO_LOCAL_MODELO_AJUSTADO, 'model_args.bin'))
        torch.save (training_args, os.path.join (DIRETORIO_LOCAL_MODELO_AJUSTADO, 'training_args.bin'))

In [21]:
salvaModelo(model_args)

Vamos verificar os tamanhos dos arquivos.

In [22]:
if model_args.salvar_modelo:
  !ls -l --block-size=K /content/modelo_ajustado/

O maior arquivo é o peso do modelo, em torno de 416MB o base e 1.25G o large.

In [23]:
if model_args.salvar_modelo:
  !ls -l --block-size=M /content/modelo_ajustado/pytorch_model.bin

Para salvar seu modelo nas sessões do Colab Notebook, faça o download no seu computador local ou, idealmente, copie-o no seu Google Drive.

In [24]:
if model_args.salvar_modelo:

  # Importando as bibliotecas.
  import os

  # Import de bibliotecas
  from bert.bertmodulo import *

  # Verifica o nome do modelo BERT a ser utilizado
  MODELO_BERT = getNomeModeloBERT(model_args)

  # Verifica o tamanho do modelo(default large)
  TAMANHO_BERT =  getTamanhoBERT(model_args)
  
  # Diretório local de salvamento do modelo.
  DIRETORIO_LOCAL_MODELO_AJUSTADO = '/content/modelo_ajustado/'

  # Diretório remoto de salvamento do modelo.  
  DIRETORIO_REMOTO_MODELO_AJUSTADO = "/content/drive/MyDrive/Colab Notebooks/Data/CSTNEWS/validacao_classificacao/kfold/modelo/modelo" + MODELO_BERT + TAMANHO_BERT

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_REMOTO_MODELO_AJUSTADO):  
    # Cria o diretório
    os.makedirs(DIRETORIO_REMOTO_MODELO_AJUSTADO)
    print('Diretório criado: {}'.format(DIRETORIO_REMOTO_MODELO_AJUSTADO))
  else:
    print('Diretório já existe: {}'.format(DIRETORIO_REMOTO_MODELO_AJUSTADO))

  ## Copia o arquivo do modelo para o diretório no Google Drive.
  !cp -r '$DIRETORIO_LOCAL_MODELO_AJUSTADO'* '$DIRETORIO_REMOTO_MODELO_AJUSTADO'

  print("Modelo copiado!")

## 6.3 Tempo final de processamento



In [25]:
 # Pega o tempo atual menos o tempo do início do processamento.
finalProcessamento = time.time()
tempoTotalProcessamento = formataTempo(finalProcessamento - inicioProcessamento)

print("")
print("  Tempo processamento: {:} (h:mm:ss)".format(tempoTotalProcessamento))


  Tempo processamento: 2:51:13 (h:mm:ss)


Executa o wandb para finalizar a execução anterior

In [26]:
if model_args.use_wandb:
    # Importando a biblioteca.
    import wandb
    
    # Inicializando o registro do experimento.
    # Na execução só pode existir de um init  para que não gere dois registros no wandb.
    wandb.init(project="ajustefinocstnews_v1_c_sb_kfold", name=training_args.output_dir + str(model_args.fold))